# Preparing the specialities

so that later there is a mapping table

In [ ]:
import pandas as pd
import pdfplumber

must start with CVK CCM CBF CBB

In [ ]:
pdf_path = 'data/specialities/Kostenstellen OE.pdf'

tables = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        table = page.extract_table()
        if table:
            tables.append(table)

# Combine tables if needed
combined_table = pd.concat([pd.DataFrame(table[1:], columns=table[0]) for table in tables], ignore_index=True)

In [ ]:
# Select rows with empty 'OE' values
empty_OE_rows = combined_table[combined_table['OE'] == ""]

# Select rows with non-empty 'OE' values
non_empty_OE_rows = combined_table[combined_table['OE'] != ""]

# Remove duplicates from non-empty 'OE' rows
non_empty_OE_rows = non_empty_OE_rows.drop_duplicates(subset=['OE'])

# Combine both dataframes back together
combined_table = pd.concat([empty_OE_rows, non_empty_OE_rows])

combined_table = combined_table.sort_index()

combined_table = combined_table.reset_index(drop=True)

In [ ]:
# count rows in which 'CC' is empty

# if >=2:
# - take the second last row as title
# - save all following OEs
# - do this until there are again at least 2 empty rows
# - the OEs then go into the dataset


# Initialize list to collect data
specialities_data = []

currentlyAddingOEs = False

currentSpeciality = None
currentOEs = []

for index, row in combined_table.iterrows():
    cc = row['CC']
    oe = row['OE']

    if (cc == "" and oe == ""):
        currentlyAddingOEs = False
        continue

    if not currentlyAddingOEs:
        currentlyAddingOEs = True

        cc_line_before = combined_table.at[index-1, 'CC']
        cc_line_2_before = combined_table.at[index-2, 'CC']
        oe_line_before = combined_table.at[index-1, 'OE']
        oe_line_2_before = combined_table.at[index-2, 'OE']

        if (cc_line_before == "") and (cc_line_2_before == "") and (oe_line_before == "") and (oe_line_2_before == ""):
            if currentSpeciality is not None:
                specialities_data.append({'speciality': currentSpeciality, 'OEs': currentOEs})
            currentSpeciality = combined_table.at[index-2, 'KOSTENSTELLENSTRUKTUR']
            currentOEs = []

    if oe !="":
        currentOEs.append(oe)

# Handle the last group
specialities_data.append({'speciality': currentSpeciality, 'OEs': currentOEs})

# Create DataFrame from accumulated data
specialities = pd.DataFrame(specialities_data)

# Now 'specialities' DataFrame contains the collected data


In [ ]:
specialities = specialities[specialities['OEs'].apply(lambda x: len(x) > 0)]


In [ ]:
#remove all campus occurences
specialities['speciality'] = specialities['speciality'].str.replace(r'\b(?:CCM|CVK|CBF|CBB|FL|DL)\b/?', '', regex=True)
#remove all multible whitespaces
specialities['speciality'] = specialities['speciality'].str.replace(r'\s+', ' ', regex=True)
# remove all leading commas
specialities['speciality'] = specialities['speciality'].str.lstrip(',')
# remove all leading dots
specialities['speciality'] = specialities['speciality'].str.lstrip('.')
# remove all leading and trailing whitespaces
specialities['speciality'] = specialities['speciality'].str.strip()

## take out everithing that connects data to the hospital


In [ ]:
# taken out to ensure data privacy

## add manual specialities

In [ ]:
manuals = pd.read_csv('data/specialities/manual_specialities.csv', sep=';')

In [ ]:
def string_to_array(x):
    return [x]
manuals['OEs'] = manuals['OEs'].apply(string_to_array)

In [ ]:
specialities = pd.concat([specialities, manuals], ignore_index=True)

## and going on with the processing...

In [ ]:
# Define a function to concatenate lists
def merge_OEs(list_of_lists):
    merged = []
    for sublist in list_of_lists:
        merged.extend(sublist)
    return merged

# Group by 'speciality' and aggregate
specialities = specialities.groupby('speciality').agg({'OEs': merge_OEs}).reset_index()

In [ ]:
specialities = specialities.sort_values('speciality').reset_index(drop=True)

In [ ]:
pd.set_option('display.max_rows', None)  # Adjust the number as needed
pd.set_option('display.max_columns', None)  # Adjust the number as needed


# change speciality names here

In [ ]:
specialities['speciality'].to_csv('data/out/specialities.csv', index=False, header=False)

In [ ]:
specialities_exploded = specialities.explode('OEs')

In [ ]:
specialities_exploded.to_pickle("data/specialities/specialities.pkl")
specialities_exploded.to_csv("data/specialities/specialities.csv", index=False, sep=';')

# adjust mapping


In [ ]:
mapping_table = pd.read_csv('data/specialities/speciality_mapping.csv', sep=';')
maual_mapping_table = pd.read_csv('data/specialities/manual_speciality_mapping.csv', sep=';')
mapping_table = pd.concat([mapping_table, maual_mapping_table], ignore_index=True)
mapping_table.to_pickle("data/specialities/speciality_mapping.pkl")